In [1]:
#load libraries
import pandas as pd
import os as os
import numpy as np
import re
import ipywidgets as widgets
from ipywidgets import interactive
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

## Crawled database from https://www.henleypassportindex.com

In [2]:
# scrapping script here: https://github.com/jeremielamboley/world-passport-power-rank-enriched/blob/master/henley_passport_index_data_collection.ipynb
df_visa_free = pd.read_csv('https://raw.githubusercontent.com/jeremielamboley/world-passport-power-rank-enriched/master/henley_visa_free_11_01_2019.csv', encoding = "ISO-8859-1")

In [3]:
df_visa_free.rename(columns={'passport': 'passportfrom_alpha_2code'}, inplace=True)
df_visa_free.rename(columns={'to': 'countryto_alpha_2code'}, inplace=True)

In [4]:
df_visa_free.shape

(45173, 3)

In [5]:
df_visa_free['passportfrom_alpha_2code'].nunique()

198

In [6]:
df_visa_free['countryto_alpha_2code'].nunique()

226

In [7]:
df_visa_free.head(5)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree
0,AM,AF,0
1,AM,AL,1
2,AM,DZ,0
3,AM,AS,0
4,AM,AD,0


## List of world countries with ISO codes
### Source Wikipedia https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes
### Project by Radcliff https://gist.github.com/radcliff/f09c0f88344a7fcef373

In [8]:
df_iso = pd.read_csv('https://gist.githubusercontent.com/radcliff/f09c0f88344a7fcef373/raw/2753c482ad091c54b1822288ad2e4811c021d8ec/wikipedia-iso-country-codes.csv', encoding = "utf8")

In [9]:
df_iso.rename(columns={'English short name lower case': 'Country name'}, inplace=True)
df_iso.rename(columns={'Alpha-2 code': 'country_alpha_2code'}, inplace=True)
df_iso.rename(columns={'Alpha-3 code': 'country_alpha_3code'}, inplace=True)
df_iso.rename(columns={'Numeric code': 'country_numeric_code'}, inplace=True)
df_iso = df_iso.drop(['ISO 3166-2'], axis=1)

In [10]:
df_iso.shape

(246, 4)

In [11]:
df_iso.sample(5)

,Country name,country_alpha_2code,country_alpha_3code,country_numeric_code
164,Norway,NO,NOR,578
79,Georgia,GE,GEO,268
76,French Southern Territories,TF,ATF,260
184,"Saint Helena, Ascension and Tristan da Cunha",SH,SHN,654
130,Madagascar,MG,MDG,450


## GDP per countries in USD, 2016, from the United Nations Statistics Division
### Source: http://data.un.org/

In [12]:
df_GDP_UN = pd.read_csv('http://data.un.org/_Docs/SYB/CSV/SYB61_T13_GDP%20and%20GDP%20Per%20Capita.csv', 
                        encoding = "ISO-8859-1", 
                        delimiter = ',',
                        header=1)

In [13]:
# filter the data to GDP in 2016
df_GDP_UN = df_GDP_UN[df_GDP_UN['Series'] == 'GDP in current prices (millions of US dollars)']
df_GDP_UN = df_GDP_UN[df_GDP_UN.Year == 2016]

In [14]:
# Organize the data
df_GDP_UN['Value'] = df_GDP_UN['Value'].str.replace(',', '')
df_GDP_UN['Value'] = df_GDP_UN['Value'].astype(int)
# Renaming columns
df_GDP_UN.rename(columns={'Region/Country/Area': 'country_numeric_code'}, inplace=True)
df_GDP_UN.rename(columns={'Value': 'UN_GDP_currentPrices2016'}, inplace=True)
# Convert GDP from millions to billions
df_GDP_UN['UN_GDP_currentPrices2016_billions'] = df_GDP_UN['UN_GDP_currentPrices2016'] / 1000

In [15]:
df_GDP_UN.shape

(242, 8)

In [16]:
df_GDP_UN.sample(5)

,country_numeric_code,Unnamed: 1,Year,Series,UN_GDP_currentPrices2016,Footnotes,Source,UN_GDP_currentPrices2016_billions
1309,112,Belarus,2016,GDP in current prices (millions of US dollars),47408,NaN,"United Nations Statistics Division, New York, ...",47.408
5257,670,Saint Vincent & Grenadines,2016,GDP in current prices (millions of US dollars),765,NaN,"United Nations Statistics Division, New York, ...",0.765
3257,356,India,2016,GDP in current prices (millions of US dollars),2259642,NaN,"United Nations Statistics Division, New York, ...",2259.642
3509,400,Jordan,2016,GDP in current prices (millions of US dollars),38655,NaN,"United Nations Statistics Division, New York, ...",38.655
4873,600,Paraguay,2016,GDP in current prices (millions of US dollars),27165,NaN,"United Nations Statistics Division, New York, ...",27.165


In [17]:
df_GDP_UN = df_GDP_UN[['country_numeric_code','UN_GDP_currentPrices2016_billions']]

### World total GDP for 2016 to get a % of coverage per passport

In [18]:
# country code = 1 for the world aggregate
df_GDP_UN['UN_World_GDP_percent'] = df_GDP_UN['UN_GDP_currentPrices2016_billions'] / df_GDP_UN[df_GDP_UN['country_numeric_code'] == 1].iloc[0,1]

In [19]:
df_GDP_UN.sample(5)

,country_numeric_code,UN_GDP_currentPrices2016_billions,UN_World_GDP_percent
506,34,3317.296,0.043851
4217,583,0.330,0.000004
450,30,18489.486,0.244412
5341,678,0.343,0.000005
62,15,700.350,0.009258


# Population and surface, 2015, from the United Nations Statistics Division

### Source: http://data.un.org/

In [20]:
df_population_surface_UN = pd.read_csv('http://data.un.org/_Docs/SYB/CSV/SYB61_T02_Population,%20Surface%20Area%20and%20Density.csv', 
                        encoding = "ISO-8859-1", 
                        delimiter = ',',
                        header=1)

In [21]:
# filter the data to GDP in 2016
df_population_surface_UN = df_population_surface_UN[(df_population_surface_UN['Series'] == 'Population mid-year estimates (millions)') | (df_population_surface_UN['Series'] == 'Surface area (thousand km2)')]
df_population_surface_UN = df_population_surface_UN[df_population_surface_UN['Year'] == 2015]

In [22]:
# Organize the data
df_population_surface_UN['Value'] = df_population_surface_UN['Value'].str.replace(',', '')
df_population_surface_UN['Value'] = df_population_surface_UN['Value'].astype(float)

In [23]:
df_population_surface_UN = df_population_surface_UN[['Region/Country/Area','Series','Value']]
df_population_surface_UN.rename(columns={'Region/Country/Area': 'country_numeric_code'}, inplace=True)

In [24]:
# pivot and un-pivot
df_population_surface_UN = df_population_surface_UN.pivot(index='country_numeric_code', columns='Series', values='Value')
df_population_surface_UN = pd.DataFrame(df_population_surface_UN.to_records())

In [25]:
# get the world total for population and surface
df_population_UN_world = df_population_surface_UN[(df_population_surface_UN.country_numeric_code == 1)].iloc[0,1]
df_surface_UN_world = df_population_surface_UN[(df_population_surface_UN.country_numeric_code == 1)].iloc[0,2]

In [26]:
# World Population mid-year estimates 7383 millions, 2015
# Source: United Nations Statistics Division, New York, World Population Prospects: The 2017 Revision, last accessed June 2017.
df_population_surface_UN['UN_population_Value_2015_percent'] = (df_population_surface_UN['Population mid-year estimates (millions)'] / df_population_UN_world)

In [27]:
df_population_surface_UN['Surface_area_convered_percent'] = (df_population_surface_UN['Surface area (thousand km2)'] / df_surface_UN_world)

In [28]:
df_population_surface_UN.sample(5)

,country_numeric_code,Population mid-year estimates (millions),Surface area (thousand km2),UN_population_Value_2015_percent,Surface_area_convered_percent
154,474,0.39,1.0,0.000053,0.000007
36,57,0.52,3.0,0.000070,0.000022
88,231,99.87,1104.0,0.013527,0.008108
58,140,4.55,623.0,0.000616,0.004575
193,624,1.77,36.0,0.000240,0.000264


## Intersect df_visa_free and df_iso on 'country to'

In [29]:
# copy df_visa_free, rename the columns from centadata and hauntedhouse
df_visa_free2 = df_visa_free.copy()

In [30]:
df_visa_free2.rename(columns={'countryto_alpha_2code': 'country_alpha_2code'}, inplace=True)

In [31]:
# Intersect on 'countryto_alpha_2code'
df = pd.merge(df_visa_free2, df_iso, on=['country_alpha_2code'], how='left')

In [32]:
df.rename(columns={'country_alpha_2code': 'countryto_alpha_2code'}, inplace=True)
df.rename(columns={'Country name': 'countryto_Country name'}, inplace=True)
df.rename(columns={'country_alpha_3code': 'countryto_country_alpha_3code'}, inplace=True)
df.rename(columns={'country_numeric_code': 'countryto_country_numeric_code'}, inplace=True)

In [33]:
df['passportfrom_alpha_2code'].nunique()

198

In [34]:
df['countryto_alpha_2code'].nunique()

226

In [35]:
df.sample(10)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code
38216,BB,CA,1,Canada,CAN,124.0
27435,LU,MU,1,Mauritius,MUS,480.0
15663,ZA,PW,1,Palau,PLW,585.0
37183,BB,BJ,1,Benin,BEN,204.0
14010,NG,AW,0,Aruba,ABW,533.0
6771,UY,IQ,0,Iraq,IRQ,368.0
30702,ML,UG,1,Uganda,UGA,800.0
32034,HK,IL,1,Israel,ISR,376.0
21536,PH,TN,0,Tunisia,TUN,788.0
1129,AD,AL,1,Albania,ALB,8.0


## Intersect df_visa_free and df_iso on 'passport'

In [36]:
df_iso.rename(columns={'country_alpha_2code': 'passportfrom_alpha_2code'}, inplace=True)

In [37]:
# Intersect on 'passportfrom_alpha_2code'
df = pd.merge(df, df_iso, on=['passportfrom_alpha_2code'], how='left')

In [38]:
df_iso.rename(columns={'passportfrom_alpha_2code': 'country_alpha_2code'}, inplace=True)

In [39]:
df.rename(columns={'Country name': 'passportfrom_Country name'}, inplace=True)

In [40]:
df = df.drop(['country_alpha_3code'], axis=1)
df = df.drop(['country_numeric_code'], axis=1)

## Intersect df and df_GDP_UN

In [41]:
df_GDP_UN2 = df_GDP_UN.copy()
df_GDP_UN2.rename(columns={'country_numeric_code': 'countryto_country_numeric_code'}, inplace=True)

In [42]:
df2 = pd.merge(df, df_GDP_UN2, on=['countryto_country_numeric_code'], how='left')

In [43]:
df2.rename(columns={'UN_GDP_currentPrices2016_billions': 'countryto_UN_GDP_currentPrices2016'}, inplace=True)

In [44]:
df2.sample(5)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,passportfrom_Country name,countryto_UN_GDP_currentPrices2016,UN_World_GDP_percent
3301,AL,CG,0,Congo,COG,178.0,Albania,7.778,0.000103
27433,MN,TH,1,Thailand,THA,764.0,Mongolia,407.026,0.005380
34137,HK,MA,1,Morocco,MAR,504.0,Hong Kong,103.607,0.001370
29389,GM,VG,1,"Virgin Islands, British",VGB,92.0,Gambia,0.971,0.000013
38458,DO,JM,0,Jamaica,JAM,388.0,Dominican Republic,14.057,0.000186


## Intersect df2 and df_population_UN

In [45]:
df_population_UN2 = df_population_surface_UN.copy()
df_population_UN2.rename(columns={'country_numeric_code': 'countryto_country_numeric_code'}, inplace=True)

In [46]:
df3 = pd.merge(df2, df_population_UN2, on=['countryto_country_numeric_code'], how='left')

In [47]:
#df3.rename(columns={'UN_population_Value_2015_millions': 'countryto_UN_population_Value_2015_millions'}, inplace=True)
df3.rename(columns={'Population mid-year estimates (millions)': 'countryto_UN_population_Value_2015_millions'}, inplace=True)
df3.rename(columns={'Surface area (thousand km2)': 'countryto_Surface area (thousand km2)'}, inplace=True)

In [48]:
df3.sample(5)

,passportfrom_alpha_2code,countryto_alpha_2code,visafree,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,passportfrom_Country name,countryto_UN_GDP_currentPrices2016,UN_World_GDP_percent,countryto_UN_population_Value_2015_millions,countryto_Surface area (thousand km2),UN_population_Value_2015_percent,Surface_area_convered_percent
16753,SO,PE,0,Peru,PER,604.0,Somalia,192.210,0.002541,31.38,1285.0,0.004250,0.009437
8216,TV,KG,0,Kyrgyzstan,KGZ,417.0,Tuvalu,6.551,0.000087,5.87,200.0,0.000795,0.001469
4537,TO,DZ,0,Algeria,DZA,12.0,Tonga,159.049,0.002102,39.87,2382.0,0.005400,0.017494
6710,ZM,KR,0,"Korea, Republic of",KOR,410.0,Zambia,1411.246,0.018655,50.59,100.0,0.006852,0.000734
24511,NR,SN,0,Senegal,SEN,686.0,Nauru,14.605,0.000193,14.98,197.0,0.002029,0.001447


## Reordering the columns

In [49]:
df3 = df3.reset_index(drop=True)
cols = df3.columns.tolist()
cols

['passportfrom_alpha_2code',
 'countryto_alpha_2code',
 'visafree',
 'countryto_Country name',
 'countryto_country_alpha_3code',
 'countryto_country_numeric_code',
 'passportfrom_Country name',
 'countryto_UN_GDP_currentPrices2016',
 'UN_World_GDP_percent',
 'countryto_UN_population_Value_2015_millions',
 'countryto_Surface area (thousand km2)',
 'UN_population_Value_2015_percent',
 'Surface_area_convered_percent']

In [50]:
df3 = df3[['passportfrom_alpha_2code','passportfrom_Country name','countryto_alpha_2code','countryto_Country name','countryto_country_alpha_3code', 'countryto_country_numeric_code','countryto_UN_GDP_currentPrices2016','UN_World_GDP_percent', 'countryto_UN_population_Value_2015_millions','UN_population_Value_2015_percent','countryto_Surface area (thousand km2)','Surface_area_convered_percent','visafree']]

In [51]:
df3['passportfrom_alpha_2code'].nunique()

198

In [52]:
df3.head(5)

,passportfrom_alpha_2code,passportfrom_Country name,countryto_alpha_2code,countryto_Country name,countryto_country_alpha_3code,countryto_country_numeric_code,countryto_UN_GDP_currentPrices2016,UN_World_GDP_percent,countryto_UN_population_Value_2015_millions,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),Surface_area_convered_percent,visafree
0,AM,Armenia,AF,Afghanistan,AFG,4.0,20.235,0.000267,33.74,0.004570,653.0,0.004796,0
1,AM,Armenia,AL,Albania,ALB,8.0,11.864,0.000157,2.92,0.000396,29.0,0.000213,1
2,AM,Armenia,DZ,Algeria,DZA,12.0,159.049,0.002102,39.87,0.005400,2382.0,0.017494,0
3,AM,Armenia,AS,American Samoa,ASM,16.0,NaN,NaN,0.06,0.000008,0.0,0.000000,0
4,AM,Armenia,AD,Andorra,AND,20.0,2.858,0.000038,0.08,0.000011,0.0,0.000000,0


## Pandas pivot_table

In [53]:
df_pivot = pd.pivot_table(df3[df3.visafree == 1],index='passportfrom_Country name',values=['countryto_UN_GDP_currentPrices2016','UN_World_GDP_percent','countryto_UN_population_Value_2015_millions','UN_population_Value_2015_percent','countryto_Surface area (thousand km2)','Surface_area_convered_percent','visafree'], aggfunc=np.sum, margins=False,dropna=True)
df_pivot.sort_values('visafree', ascending=False)
df_pivot.head(5)

,Surface_area_convered_percent,UN_World_GDP_percent,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,visafree
passportfrom_Country name,,,,,,,
Afghanistan,0.038432,0.005828,0.048784,5233.0,440.912,360.17,30
Albania,0.235712,0.304555,0.269138,32095.0,23039.214,1987.05,115
Algeria,0.108540,0.038378,0.126856,14779.0,2903.233,936.58,50
Andorra,0.541803,0.758468,0.424182,73773.0,57377.264,3131.74,169
Angola,0.113872,0.042810,0.149718,15505.0,3238.540,1105.37,49


## Turn the pivot table in a Data frame

In [54]:
df_pivot_flattened = pd.DataFrame(df_pivot.to_records())
df_pivot_flattened.sample(5)

,passportfrom_Country name,Surface_area_convered_percent,UN_World_GDP_percent,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,visafree
82,Israel,0.537213,0.466467,0.316500,73148.0,35287.730,2336.72,161
109,Mali,0.136161,0.046452,0.179423,18540.0,3514.032,1324.68,55
195,Zambia,0.119791,0.051992,0.159574,16311.0,3933.123,1178.14,68
145,Russian Federation,0.338450,0.176707,0.332220,46084.0,13367.705,2452.78,119
22,Botswana,0.127525,0.097575,0.175632,17364.0,7381.429,1296.69,82


In [55]:
df_pivot_flattened['Surface_Rank'] = df_pivot_flattened['countryto_Surface area (thousand km2)'].rank(method='min', ascending=False).astype(int)
df_pivot_flattened['GDP_Rank'] = df_pivot_flattened['countryto_UN_GDP_currentPrices2016'].rank(method='min', ascending=False).astype(int)
df_pivot_flattened['Population_Rank'] = df_pivot_flattened['countryto_UN_population_Value_2015_millions'].rank(method='min', ascending=False).astype(int)
df_pivot_flattened['visafree_countries_Rank'] = df_pivot_flattened['visafree'].rank(method='min', ascending=False).astype(int)

## Calculate the overall ranking

In [56]:
df_pivot_flattened['Overall_ranking_calculation'] = df_pivot_flattened[['visafree_countries_Rank','GDP_Rank','Surface_Rank','Population_Rank']].sum(axis=1)

In [57]:
df_pivot_flattened['New_Overall_ranking'] = df_pivot_flattened['Overall_ranking_calculation'].rank(method='min', ascending=True).astype(int)

## Create a new column Gain/loss in ranking

In [58]:
df_pivot_flattened['gain-loss'] = df_pivot_flattened['visafree_countries_Rank'] - df_pivot_flattened['New_Overall_ranking']

In [59]:
df_pivot_flattened.sample(5)

,passportfrom_Country name,Surface_area_convered_percent,UN_World_GDP_percent,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),countryto_UN_GDP_currentPrices2016,countryto_UN_population_Value_2015_millions,visafree,Surface_Rank,GDP_Rank,Population_Rank,visafree_countries_Rank,Overall_ranking_calculation,New_Overall_ranking,gain-loss
29,Cameroon,0.102261,0.028978,0.124570,13924.0,2192.162,919.70,48,170,182,174,173,699,176,-3
121,Mozambique,0.115084,0.047345,0.151162,15670.0,3581.587,1116.03,58,155,160,160,143,618,158,-15
0,Afghanistan,0.038432,0.005828,0.048784,5233.0,440.912,360.17,30,197,197,195,196,785,197,-1
37,Comoros,0.101387,0.047881,0.154298,13805.0,3622.137,1139.18,52,173,157,158,163,651,163,0
47,Djibouti,0.080434,0.030282,0.114789,10952.0,2290.806,847.49,45,184,178,177,179,718,181,-2


## Reorder the columns

In [60]:
df_pivot_flattened = df_pivot_flattened[['passportfrom_Country name',
                                         'New_Overall_ranking',
                                         'gain-loss',
                                         'visafree',
                                         'visafree_countries_Rank',
                                         'countryto_UN_GDP_currentPrices2016',
                                         'GDP_Rank',
                                         'UN_World_GDP_percent',
                                         'countryto_UN_population_Value_2015_millions',
                                         'Population_Rank',
                                         'UN_population_Value_2015_percent',
                                         'countryto_Surface area (thousand km2)',
                                         'Surface_Rank',
                                         'Surface_area_convered_percent',
                                         'Overall_ranking_calculation']]

In [61]:
# Re-order and format the values
df_pivot_flattened.sort_values('New_Overall_ranking',ascending=True, inplace=True)
df_pivot_flattened = df_pivot_flattened.reset_index(drop=True)
df_pivot_flattened.style.format({
    'UN_World_GDP_percent': '{:.2%}'.format,
    'UN_population_Value_2015_percent': '{:.2%}'.format,
    'Surface_area_convered_percent': '{:.2%}'.format
    })

,passportfrom_Country name,New_Overall_ranking,gain-loss,visafree,visafree_countries_Rank,countryto_UN_GDP_currentPrices2016,GDP_Rank,UN_World_GDP_percent,countryto_UN_population_Value_2015_millions,Population_Rank,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),Surface_Rank,Surface_area_convered_percent,Overall_ranking_calculation
0,Singapore,1,1,189,2,68781.9,1,90.92%,4746.51,3,64.29%,90971,2,66.81%,8
1,Japan,2,-1,190,1,64790.1,4,85.65%,5923.63,1,80.23%,86657,4,63.64%,10
2,"Korea, Republic of",3,-1,189,2,60116.5,6,79.47%,4826.81,2,65.38%,100305,1,73.67%,11
3,Denmark,4,2,187,6,57567.6,9,76.10%,3341.93,17,45.27%,79750,9,58.57%,41
4,Finland,5,1,187,6,57636,7,76.19%,3342.14,16,45.27%,79455,13,58.35%,42
5,Sweden,6,0,187,6,57360,15,75.82%,3337.86,18,45.21%,79354,14,58.28%,53
6,Luxembourg,7,3,186,10,57610.6,8,76.16%,3253.48,26,44.07%,79459,12,58.36%,56
7,San Marino,8,38,168,46,68520.1,2,90.58%,4481.91,4,60.71%,82795,7,60.81%,59
8,Brunei Darussalam,9,39,165,48,65240,3,86.24%,4237.51,5,57.40%,84063,6,61.74%,62
9,Italy,10,-4,187,6,56015.6,28,74.05%,3288.12,22,44.54%,79491,11,58.38%,67


In [62]:
# Export to CSV
df_pivot_flattened.to_csv('ranking.csv')

## Biggest winners and losers

In [63]:
df_rankgainloss = df_pivot_flattened.sort_values('gain-loss', ascending=False, inplace=False)
df_rankgainloss.style.format({
    'UN_World_GDP_percent': '{:.2%}'.format,
    'UN_population_Value_2015_percent': '{:.2%}'.format,
    'Surface_area_convered_percent': '{:.2%}'.format
    })

,passportfrom_Country name,New_Overall_ranking,gain-loss,visafree,visafree_countries_Rank,countryto_UN_GDP_currentPrices2016,GDP_Rank,UN_World_GDP_percent,countryto_UN_population_Value_2015_millions,Population_Rank,UN_population_Value_2015_percent,countryto_Surface area (thousand km2),Surface_Rank,Surface_area_convered_percent,Overall_ranking_calculation
138,Nepal,139,49,40,188,4964.39,133,6.56%,2448.87,73,33.17%,14385,166,10.56%,560
59,Ecuador,60,39,93,99,23156.8,90,30.61%,3379.38,15,45.77%,63280,45,46.47%,249
8,Brunei Darussalam,9,39,165,48,65240,3,86.24%,4237.51,5,57.40%,84063,6,61.74%,62
7,San Marino,8,38,168,46,68520.1,2,90.58%,4481.91,4,60.71%,82795,7,60.81%,59
76,Belarus,76,36,77,112,20372.7,98,26.93%,3191.17,31,43.22%,61245,51,44.98%,292
57,Bosnia and Herzegovina,58,33,118,91,35374.1,60,46.76%,3485.16,13,47.21%,59135,61,43.43%,225
129,Lao People's Democratic Republic,130,33,52,163,5365.78,131,7.09%,1485.53,116,20.12%,33337,108,24.48%,518
25,"Taiwan, Province of China",26,32,149,58,62452.6,5,82.56%,3868.61,8,52.40%,64458,44,47.34%,115
45,Serbia,46,30,129,76,41387.1,47,54.71%,3583.12,12,48.53%,67113,40,49.29%,175
71,Fiji,72,29,89,101,22117.1,94,29.24%,3272,23,44.32%,57109,67,41.94%,285


# A little program to access the results easily

In [64]:
# create a new list
countrylist = list(set(df_pivot_flattened['passportfrom_Country name'].drop_duplicates().values.tolist()))
countrylist.sort()

In [65]:
# Select the passport holder's country
countryselected = widgets.Dropdown(
    options=countrylist,
    value='Hong Kong',
    description="Country:",
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print ("Country selected: %s" % change['new'])

countryselected.observe(on_change)
display(countryselected)

Dropdown(description='Country:', index=74, options=('Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', …

In [66]:
# Selected country in the drop down menu
countryselectedvisafree = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['visafree']
countryselectedvisafree = countryselectedvisafree.to_frame()
countryselectedvisafree = countryselectedvisafree.iloc[0]['visafree']
# the overall country rank
countryrank = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['New_Overall_ranking']
countryrank = countryrank.to_frame()
countryrank = countryrank.iloc[0]['New_Overall_ranking']
# the gain-loss compare the to Henley Passport Index 
gainloss = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['gain-loss']
gainloss = gainloss.to_frame()
gainloss = gainloss.iloc[0]['gain-loss']
# the rank regarding the number of countries accessible visa-free
visafreecountriesRank = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['visafree_countries_Rank']
visafreecountriesRank = visafreecountriesRank.to_frame()
visafreecountriesRank = visafreecountriesRank.iloc[0]['visafree_countries_Rank']
# the rank regarding to the population accessible visa-free
countryselectedpopulation = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['UN_population_Value_2015_percent']
countryselectedpopulation = countryselectedpopulation.to_frame()
countryselectedpopulation = countryselectedpopulation.iloc[0]['UN_population_Value_2015_percent']
# the rank regarding to the globe surface accessible visa-free
countryselectedsurface = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['Surface_area_convered_percent']
countryselectedsurface = countryselectedsurface.to_frame()
countryselectedsurface = countryselectedsurface.iloc[0]['Surface_area_convered_percent']
countryselectedsurface = countryselectedsurface.item()
# the rank regarding to the world GDP accessible visa-free
countryselectedGDP = df_pivot_flattened[df_pivot_flattened['passportfrom_Country name'] == countryselected.value]['UN_World_GDP_percent']
countryselectedGDP = countryselectedGDP.to_frame()
countryselectedGDP = countryselectedGDP.iloc[0]['UN_World_GDP_percent']

In [67]:
# Run this query to get the statistics
def country_access(countryselected,countryselectedvisafree,countryrank,visafreecountriesRank,gainloss,countryselectedpopulation,countryselectedsurface,countryselectedGDP):
    print("With " + countryselected.value + "'s passport you have access to:")
    print(str(countryselectedvisafree) + " countries visa-free, ranking " + str(countryrank)  +  " over " + str(df_pivot_flattened['passportfrom_Country name'].nunique()) + " countries' passports according to our enriched ranking.")
    print("Ranked " + str(visafreecountriesRank) + " according to Henley Passport Index. " + str(np.where(gainloss < 0, "Rank loss: "+str(gainloss), "Rank gain: +" + str(gainloss))))
    print("\nIt gives you visa-free access to:")
    print("\t" + "{:.0%}".format(countryselectedpopulation) + " of the world's population")
    print("\t" + "{:.0%}".format(countryselectedsurface) + " of the world's surface")
    print("\t" + "{:.0%}".format(countryselectedGDP) + " of the world's GPD")
    
country_access(countryselected,countryselectedvisafree,countryrank,visafreecountriesRank,gainloss,countryselectedpopulation,countryselectedsurface,countryselectedGDP)

With Hong Kong's passport you have access to:
169 countries visa-free, ranking 37 over 197 countries' passports according to our enriched ranking.
Ranked 42 according to Henley Passport Index. Rank gain: +5

It gives you visa-free access to:
	40% of the world's population
	65% of the world's surface
	52% of the world's GPD
